<a href="https://colab.research.google.com/github/Sujal-vajire/Machine-learning-models/blob/main/2DTreeGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vtk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 MB 8.9 MB/s eta 0:00:00


In [4]:
pip install -e git+https://github.com/likchuan/vtk_py.git#egg=vtk_py


Obtaining vtk_py from git+https://github.com/likchuan/vtk_py.git#egg=vtk_py
  Cloning https://github.com/likchuan/vtk_py.git to ./src/vtk-py
  Running command git clone --filter=blob:none --quiet https://github.com/likchuan/vtk_py.git /content/src/vtk-py
  Resolved https://github.com/likchuan/vtk_py.git to commit cc4df559e69187ee2d947bf1b10800413871b510
ERROR: vtk_py from git+https://github.com/likchuan/vtk_py.git#egg=vtk_py does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [6]:
import turtle
import numpy as np
from matplotlib import pylab as plt
import math
import vtk as vtk
import sys
sys.path.append("/mnt/Research")
import vtk_py as vtk_py
from vtk.util.numpy_support import numpy_to_vtk
from vtk.util.numpy_support import vtk_to_numpy
from createTreeVTK import *
from random import seed
from random import random
import os
from treegenlib import *

def main():
    nterm = 200  # 500
    dmin = 0.5
    Qperf = 1
    Pperf = 100
    Pterm = 60

    # Create 1st vessel
    pts = vtk.vtkPoints()
    cells = vtk.vtkCellArray()
    pts.InsertNextPoint(0, 0, 0)
    cells, pts = createvessel(100, 0, cells, pts)

    pdata = vtk.vtkPolyData()
    pdata.SetPoints(pts)
    pdata.SetLines(cells)

    # Add radius
    radius = []
    radius.append(1)
    radius_vtk = numpy_to_vtk(np.array(radius), deep=1, array_type=vtk.VTK_FLOAT)
    radius_vtk.SetName("radius")
    pdata.GetCellData().AddArray(radius_vtk)

    # Add betaL
    betaL = []
    betaL.append(1)
    betaL_vtk = numpy_to_vtk(np.array(betaL), deep=1, array_type=vtk.VTK_FLOAT)
    betaL_vtk.SetName("betaL")
    pdata.GetCellData().AddArray(betaL_vtk)

    # Add betaR
    betaR = []
    betaR.append(1)
    betaR_vtk = numpy_to_vtk(np.array(betaR), deep=1, array_type=vtk.VTK_FLOAT)
    betaR_vtk.SetName("betaR")
    pdata.GetCellData().AddArray(betaR_vtk)

    # Add Q
    Qflow = []
    Qflow.append(Qperf)
    Qflow_vtk = numpy_to_vtk(np.array(Qflow), deep=1, array_type=vtk.VTK_FLOAT)
    Qflow_vtk.SetName("Qflow")
    pdata.GetCellData().AddArray(Qflow_vtk)

    # Add Parent
    parent = []
    parent.append(0)
    parent_vtk = numpy_to_vtk(np.array(parent), deep=1, array_type=vtk.VTK_INT)
    parent_vtk.SetName("parent")
    pdata.GetCellData().AddArray(parent_vtk)

    pdata = GetReducedHydroR(pdata)

    # Add output directory if it does not exist
    if not os.path.exists("../output"):
        os.makedirs("../output")
    else:
        os.system("rm ../output/*")

    for term in np.arange(1, nterm + 1):
        cnt = 0
        print("Finding nearest vessel")
        while (1):
            x = randomgenerator(0, 1)
            y = randomgenerator(0, 1)

            cellid, dist, closestpt = findclosestvessel([x, y, 0], pdata)

            if (dist > dmin):
                break

            if (cnt == 10):
                dmin = 0.9 * dmin
                cnt = 0
                continue

            cnt += 1

        print("Create " + str(term) + " vessel at terminal point ", x, ", ", y)
        Qterm = Qperf / (term + 1.0)

        # Compute radius of all vessels
        Rstar_root = GetSubTreeResistance(0, pdata)
        r_root = (Rstar_root * Qperf / (Pperf - Pterm)) ** (0.25)
        pdata = ComputeRadiusOfAllVessels(0, pdata, r_root)

        newpdata = addvessel(cellid, [x, y], closestpt, pdata, Qterm)

        newpdata = ComputeRadiusOfAllVessels(0, newpdata, r_root)

        Qterm = ComputeTerminalFlow


ModuleNotFoundError: ignored